In [1]:
import torch
from CSP_training import *
import matplotlib.pyplot as plt

In [2]:
data_path = Path("../../Datasets/ILSVRC/Data/")
image_path = data_path / "CLS-LOC"

#download_dataset()
#walk_through_dir(image_path)


# Setup train and testing paths
train_dir = image_path / "train"
test_dir = image_path / "val"


# Write transform for image
data_transform = transforms.Compose([
    # Resize the images to 64x64
    transforms.Resize(size=(640, 640)),
    # Flip the images randomly on the horizontal
    #transforms.RandomHorizontalFlip(p=0.5), # p = probability of flip, 0.5 = 50% chance
    #transforms.RandomRotation(degrees=0.2),
    # Turn the image into a torch.Tensor
    transforms.ToTensor() # this also converts all pixel values from 0 to 255 to be between 0.0 and 1.0 
])
# Use ImageFolder to create dataset(s)

train_data = datasets.ImageFolder(root=train_dir, # target folder of images
                                  transform=data_transform, # transforms to perform on data (images)
                                  target_transform=None) # transforms to perform on labels (if necessary)

test_data = datasets.ImageFolder(root=test_dir, 
                                 transform=data_transform,
                                 target_transform=None)

print(f"Train data:\n{train_data}\nTest data:\n{test_data}")
# Turn train and test Datasets into DataLoaders
from torch.utils.data import DataLoader
train_dataloader = DataLoader(dataset=train_data, 
                              batch_size=8, # how many samples per batch?
                              num_workers=8, # how many subprocesses to use for data loading? (higher = more)
                              shuffle=True) # shuffle the data?

test_dataloader = DataLoader(dataset=test_data, 
                             batch_size=8, 
                             num_workers=4, 
                             shuffle=False) # don't usually need to shuffle testing data

img, label = next(iter(train_dataloader))

CSP = CSPDarknet(3, 16).to("cuda:4")

img_batch, label_batch = next(iter(test_dataloader))
img_single, label_single = img_batch[0].unsqueeze(dim=0).to("cuda:4"), label_batch[0]

CSP.eval()
with torch.inference_mode():
    pred = CSP(img_single)
    
"""print(f"Output logits:\n{pred}\n")
print(f"Output prediction probabilities:\n{torch.softmax(pred, dim=1)}\n")
print(f"Output prediction label:\n{torch.argmax(torch.softmax(pred, dim=1), dim=1)}\n")
print(f"Actual label:\n{label_single}")"""

NUM_EPOCHS = 2
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=CSP.parameters(), lr=0.001)

res = train(model=CSP, train_dataloader=train_dataloader, test_dataloader=test_dataloader,
           optimizer=optimizer,
           loss_fn=loss_fn,
           epochs=NUM_EPOCHS)

Train data:
Dataset ImageFolder
    Number of datapoints: 1281169
    Root location: ../../Datasets/ILSVRC/Data/CLS-LOC/train
    StandardTransform
Transform: Compose(
               Resize(size=(640, 640), interpolation=bilinear, max_size=None, antialias=None)
               ToTensor()
           )
Test data:
Dataset ImageFolder
    Number of datapoints: 50001
    Root location: ../../Datasets/ILSVRC/Data/CLS-LOC/val
    StandardTransform
Transform: Compose(
               Resize(size=(640, 640), interpolation=bilinear, max_size=None, antialias=None)
               ToTensor()
           )


  0%|          | 0/2 [00:00<?, ?it/s]

Train step
test step


../aten/src/ATen/native/cuda/Loss.cu:242: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [7,0,0] Assertion `t >= 0 && t < n_classes` failed.


error
error
error
error
error
error
error
Epoch: 1 | train_loss: 6.9158 | train_acc: 0.0010 | test_loss: 6.9034 | test_acc: 429428.0119
Train step


RuntimeError: cuDNN error: CUDNN_STATUS_MAPPING_ERROR

In [ ]:
def plot_loss_curves(results):
    """Plots training curves of a results dictionary.

    Args:
        results (dict): dictionary containing list of values, e.g.
            {"train_loss": [...],
             "train_acc": [...],
             "test_loss": [...],
             "test_acc": [...]}
    """
    
    # Get the loss values of the results dictionary (training and test)
    loss = results['train_loss']
    test_loss = results['test_loss']

    # Get the accuracy values of the results dictionary (training and test)
    accuracy = results['train_acc']
    test_accuracy = results['test_acc']

    # Figure out how many epochs there were
    epochs = range(len(results['train_loss']))

    # Setup a plot 
    plt.figure(figsize=(15, 7))

    # Plot loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, loss, label='train_loss')
    plt.plot(epochs, test_loss, label='test_loss')
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.legend()

    # Plot accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, accuracy, label='train_accuracy')
    plt.plot(epochs, test_accuracy, label='test_accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.legend();

In [ ]:
plot_loss_curves(res)

In [ ]:
CSP.to("cuda:4").eval()

In [ ]:
torch.onnx.export(CSP, torch.randn(1, 3, 640, 640).to("cuda:4"), "batch_csp_darknet.onnx", input_names=['input'], output_names=['output'])

In [ ]:
CSP.to("cpu")

In [ ]:
import torch
import onnx
from torch.profiler import profile, record_function, ProfilerActivity
from CSPDarknet_YOLOv5_final import batch_CSPDarknet, CSPDarknet
dummy = torch.randn(1, 3, 64, 64)#.to("cuda:1")

with profile(activities=[ProfilerActivity.CPU], record_shapes=True, profile_memory=True) as prof:
    with record_function("model_inference"):
        CSP(dummy)

print(prof.key_averages().table(sort_by="cpu_memory_usage"))